In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from tensorflow import keras
from tqdm import tqdm

In [6]:
movies = pd.read_csv(r"C:\Users\vazhitenev\PycharmProjects\Contest2023\oneproj\train\movies.csv")
countries = pd.read_csv(r"C:\Users\vazhitenev\PycharmProjects\Contest2023\oneproj\train\countries.csv")
genres = pd.read_csv(r"C:\Users\vazhitenev\PycharmProjects\Contest2023\oneproj\train\genres.csv")
staff = pd.read_csv(r"C:\Users\vazhitenev\PycharmProjects\Contest2023\oneproj\train\staff.csv")
logs = pd.read_csv(r"C:\Users\vazhitenev\PycharmProjects\Contest2023\oneproj\train\logs.csv")
#удаляем из списка фильмо те, которые не были опубликованы
movies = movies[~movies['date_publication'].isnull()]
# переводим тип float to intager
logs['movie_id'] = logs['movie_id'].astype(int)
logs['duration'] = logs['duration'].astype(int)
logs['datetime'] = pd.to_datetime(logs['datetime'])

In [19]:
# Готовим фичи пользователей
users_df = logs.groupby('user_id', as_index=False).agg({'movie_id':'nunique', 'duration':'sum'}).rename(columns={'movie_id':'movie_count', 'duration':'sum_duration'})
users_df['movie_quantile']   =pd.qcut(users_df['movie_count'].rank(method='first'), q=4, precision=0)
users_df['duration_quantile']=pd.qcut(users_df['sum_duration'].rank(method='first'), q=4, precision=0)
user_cat_feats = ["movie_quantile", "duration_quantile"]

# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
  # получаем датафрейм с one-hot encoding для каждой категориальной фичи
  ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
  # конкатенируем ohe-hot датафрейм с датафреймом, 
  # который мы получили на предыдущем шаге
  users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


,user_id,"movie_quantile_(0.0, 51845.0]","movie_quantile_(51845.0, 103689.0]","movie_quantile_(103689.0, 155533.0]","movie_quantile_(155533.0, 207377.0]","duration_quantile_(0.0, 51845.0]","duration_quantile_(51845.0, 103689.0]","duration_quantile_(103689.0, 155533.0]","duration_quantile_(155533.0, 207377.0]"
0,0,0,0,0,1,0,0,0,1
1,1,0,0,0,1,0,0,0,1
2,2,0,0,0,1,0,0,0,1
3,3,0,0,0,1,0,0,0,1
4,4,0,0,0,1,0,0,0,1


In [22]:
# Готовим фичи айтемов
items_df = movies.rename(columns={'id':'item_id'})

item_cat_feats = ['year', 'genres', 'countries']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
  ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
  items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1) 

items_ohe_df.head()

,item_id,year_1895-01-01,year_1906-01-01,year_1925-01-01,year_1934-01-01,year_1937-01-01,year_1938-01-01,year_1939-01-01,year_1940-01-01,year_1941-01-01,...,"countries_[81, 121, 102, 146]","countries_[81, 121]",countries_[81],"countries_[83, 102]","countries_[83, 242, 102]",countries_[83],countries_[84],"countries_[85, 122, 121, 102]","countries_[93, 109]",countries_[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
'''
Собираем матрицу взаимодействий
В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил. 

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.

_Можете не фильтровать такие события, тогда у вас будет больше негативных примеров._
'''
interactions_df = logs.groupby(['user_id', 'movie_id'], as_index=False).agg({'datetime':'max', 'duration':'sum'}).rename(columns={'movie_id':'item_id', 'duration':'total_dur', 'datetime':'last_watch_dt'})

print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 35 процентов
# замена на 10 минут
interactions_df = interactions_df[interactions_df.total_dur > 600]

# соберем всех пользователей, которые посмотрели 
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
  if entries > 2:
    valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
  if entries > 5:
    valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 207377
N items before: 5283

N users after: 76429
N items after: 4343


In [28]:
'''
После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. 
Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.
'''
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

76429
3963


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [35]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,uid,iid
0,0,12,2023-06-12 18:28:52.927833+03:00,8690,0,4
1,0,74,2023-06-14 12:16:18.451369+03:00,36183,0,29
2,0,107,2023-06-14 18:47:34.563849+03:00,110996,0,43
4,0,165,2023-04-18 11:59:47.414031+03:00,2286,0,81
5,0,190,2023-06-05 15:35:02.474099+03:00,934,0,93


Наконец соберем и отнормируем матрицу взаимодействий:

In [36]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), 
                             interactions_df.iid.nunique())) 

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]


In [37]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique())

3963
3963
76298
76429


set()

Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари 

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [38]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid =  [["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [72]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid.get(x, 'unknown'))
users_ohe_df = users_ohe_df.set_index("uid")

# DSSM starter's pack

Возьмем вектор юзера (anchor) и векторы двух айтемов - "хорошего" и "плохого" (positive и negative). Хороший айтем - это тот, который пользователь уже посмотрел, а в качестве плохого возьмем любой случайный айтем из датасета. Затем посчитаем расстояния:
1. между вектором юзера и вектором "хорошего" айтема
2. между вектором юзера и вектором "плохого" айтема

_Значением функции потерь будет разность между первым и вторым расстоянием._

In [73]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный
 
    return loss

In [74]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0]-1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])
            
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


Посмотрим, что получается:

In [75]:
# инициализируем генератор
gen = generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                users=users_ohe_df.drop(["user_id"], axis=1), 
                interactions=interactions_vec)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 8)
вектор взаимодействий юзера с айтемами: (1024, 3963)
вектор 'хорошего' айтема: (1024, 1302)
вектор 'плохого' айтема: (1024, 1302)

вектор фичей юзера: (1024, 8)
вектор взаимодействий юзера с айтемами: (1024, 3963)


## Собираем модель

Для того, чтобы обучить модель используя триплет лосс, нам нужно получить три вектора - вектор юзера, вектор "хорошего" айтема и вектор "плохого" айтема. Для этого нам нужно две модели ("хороший" и "плохой" айтем будут семплироваться одной и той же моделью).  

Модель юзера будет иметь два входа:
- вход для фичей юзера (фичи из users_ohe_df)
- вход для вектора айтемов, которые посмотрел юзер (строка interactions_vec, которая соответствует uid конкретного юзера)

Выход модели юзера будет размерностью N_FACTORS.

У модели айтема будет один вход для фичей айтема (из items_ohe_df) и один выход также размерностью N_FACTORS.

Общая архитектура будет вот такой: 
- есть модель юзера и модель айтема
- обе модели семплируют юзер и айтем-фичи во внутреннее пространство размерностью N_FACTORS
- модель айтема семплирует два айтема - "хороший" и "плохой"
- в итоге получается три вектора размерностью N_FACTORS (вектор юзера, вектор "хорошего" айтема и вектор "плохого" айтема)
- затем полученные векторы конкатенируются, по ним считается triplet loss
- profit

Для того, чтобы собрать модель, помимо размерности внутреннего пространства, нам нужно знать еще размерность вектора юзера и вектора айтема. Зададим их сразу.

In [76]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], ) 
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (1302,)
USER_META_MODEL_SHAPE: (8,)
USER_INTERACTION_MODEL_SHAPE: (3963,)


In [77]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    
    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя, 
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)
    
    add = keras.layers.Add()([layer_1, layer_2])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)
    
    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)
    

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])
    
    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)
    
    return keras.models.Model([inp_meta, inp_interaction], out)

# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [78]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")


In [79]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

Посмотрим, что получилось:

In [80]:
# модель айтема
item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 1302)]               0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 128)                  166656    ['input_8[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 128)                  16384     ['dense_7[0][0]']             
                                                                                                  
 add_2 (Add)                 (None, 128)                  0         ['dense_7[0][0]',             
                                                                     'dense_8[0][0]']       

In [81]:
# модель юзера
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 128)                  1024      ['input_9[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 128)                  16384     ['dense_10[0][0]']            
                                                                                                  
 input_10 (InputLayer)       [(None, 3963)]               0         []                            
                                                                                            

In [82]:
# общая модель
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 3963)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 1302)]               0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 1302)]               0         []                            
                                                                                            

In [83]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id 
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                    users=users_ohe_df.drop(["user_id"], axis=1), 
                    interactions=interactions_vec,
                    batch_size=64), 
          steps_per_epoch=100, 
          epochs=30, 
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/30
100/100 [==============================] - 9s 79ms/step - loss: 0.3002 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 7s 73ms/step - loss: 0.2261 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 9s 86ms/step - loss: 0.1910 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 9s 91ms/step - loss: 0.1733 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 8s 76ms/step - loss: 0.1507 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 8s 80ms/step - loss: 0.1475 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 8s 76ms/step - loss: 0.1342 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 8s 79ms/step - loss: 0.1304 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 7s 73ms/step - loss: 0.1204 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 7s 73ms/step - loss: 0.1166 - lr: 0.0010
Epoch 11/30
100/100 [============================

# Inference

Отлично! Мы подготовили данные, собрали модель по архитектуре DSSM и обучили ее.
Теперь возьмем случайного юзера и случайный айтем. Как понять, насколько этот айтем релевантен юзеру?

Нужно:
- получить вектор айтема;
- получить вектор юзера;
- посчитать расстояние между ними.

Это расстояние и есть мера релевантности.


In [107]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid.astype(int)]
user_interaction_vec = interactions_vec[rand_uid.astype(int)]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1), 
                        np.array(user_interaction_vec).reshape(1, -1)])

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

1/1 [==============================] - 0s 74ms/step


array([[1.5424578]], dtype=float32)

In [108]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

124/124 [==============================] - 0s 2ms/step


In [109]:
top5_iids = np.argsort(dists, axis=1)[0][:20]
top5_iids

array([2935, 1510, 1822,   87, 2664, 1399, 1455, 1911,  369, 3151, 1089,
       2211, 1977,  238, 2326, 2768, 3533, 1522, 1351, 3099], dtype=int64)

Осталось конвертировать внутренние iid в ~~id здорового человека~~ item_id:

In [110]:
top5_item_ids = [iid_to_item_id[iid] for iid in top5_iids]
top5_item_ids

[5559,
 2875,
 3461,
 177,
 5042,
 2679,
 2775,
 3637,
 741,
 5956,
 2074,
 4178,
 3759,
 484,
 4370,
 5256,
 6663,
 2893,
 2590,
 5876]

In [112]:
recommended_titles = items_df.loc[items_df.item_id.isin(top5_item_ids)].name
recommended_titles

177                       Аллея кошмаров
484                    Аватар: Путь воды
741                            Под водой
2074                             Морбиус
2590                         Оленьи рога
2679                              Вечныe
2775                     Человек на Луне
2875                      Смерть на Ниле
2893                    Не говори никому
3461                     Последняя дуэль
3637                Прошлой ночью в Сохо
3759                    Дьявол в деталях
4178              Бегущий по лезвию 2049
4370      Убийство в Восточном экспрессе
5042              Хроники хищных городов
5256                        К югу от рая
5559                                Дюна
5876    Черная пантера: Ваканда навсегда
5956                  King’s Man: Начало
6663       Клаустрофобы 2: Лига выживших
Name: name, dtype: object